# Helper

In [42]:
import json
import re
import mysql.connector

from IPython.display import display, Markdown

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

def cached(func):
    cache = dict()
    
    def return_func(arg):
        if (arg in cache):
            return cache[arg]
        else:
            result = func(arg)
            cache[arg] = result
            return result
    
    return return_func

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

def getContent(entityId):
    return json.loads(query("""
        select entity_revision_field.value from entity
        join entity_revision on entity_revision.id = entity.current_revision_id
        join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        where entity.id = %s and entity_revision_field.field = "content"
    """ % entityId)[0][0])

def getType(entityId):
    return query("""
        select type.name from entity
        join type on type.id = entity.type_id
        where entity.id = %s
    """ % entityId)[0][0]

def getTrashed(entityId):
    return query("""
        select uuid.trashed from entity
        join uuid on uuid.id = entity.id
        where entity.id = %s
    """ % entityId)[0][0]

def getInjectionIds(content, e=0):
    if "plugin" in content:
        yield from getInjectionIdsEdtrIo(content, e)
    else:
        yield from getInjectionIdsLegacy(content)
    
def getInjectionIdsLegacy(content):
    regex = re.compile(r">\[[^\]]+\]\(\/(\d+)\)")
    
    for row in content:
        for column in row:
            for match in regex.finditer(column["content"]):
                yield int(match.group(1))

def getInjectionIdsEdtrIo(content, e):
    if isinstance(content, list):
        for child in content:
            yield from getInjectionIdsEdtrIo(child, e)
    if isinstance(content, dict):
        if "plugin" in content and content["plugin"] == "injection":
            try:
                yield int(content["state"][1:])
            except ValueError:
                print("Wrong state in id %s: %s" % (e, content["state"]))
        else:
            for child in content.values():
                yield from getInjectionIdsEdtrIo(child, e)

In [51]:
from json import JSONDecodeError

entityIds = querySingleton("""
    select entity.id from entity
    join uuid on uuid.id = entity.id
    join type on type.id = entity.type_id
    where type.name != "comment"
        and type.name != "video"
        and type.name != "course"
        and type.name != "input-number-exact-match-challenge"
        and type.name != "input-expression-equal-match-challenge"
        and type.name != "input-string-normalized-match-challenge"
        and type.name != "multiple-choice-right-answer"
        and type.name != "multiple-choice-wrong-answer"
        and type.name != "single-choice-right-answer"
        and type.name != "single-choice-wrong-answer"
        and entity.current_revision_id is not null
""")

entities = dict()

for entityId in entityIds:
    try:
        content = getContent(entityId)
        injectedIds = list(getInjectionIds(content, entityId))
        
        entities[entityId] = {
            "content": content,
            "id": entityId,
            "trashed": getTrashed(entityId),
            "type": getType(entityId),
            "injectedIds": injectedIds,
            "injectedInto": []
        }
    except JSONDecodeError:
        print("Error with id: %s" % entityId)
    except IndexError:
        print("Error with id: %s" % entityId)

for entity in entities.values():
    for injectedId in entity["injectedIds"]:
        if injectedId in entities:
            entities[injectedId]["injectedInto"].append(entity["id"])

Error with id: 14573
Wrong state in id 34159: 1
Error with id: 41722
Error with id: 41729
Wrong state in id 158346: 
Wrong state in id 174232: 


# Liste von gelöschten Inhalten, die woanders inkludiert werden

In [54]:
for entity in entities.values():
    if entity["trashed"] == 1 and len(entity["injectedInto"]) > 0 and any(
      i in entities and entities[i]["trashed"] == 0 for i in entity["injectedInto"]
    ):
        display(Markdown("# ID: %s" % entity["id"]))
        display(Markdown("Der Serlo-Inhalt https://serlo.org/%s ist als gelöscht markiert, wird aber in folgenden anderen Inhalten eingebunden:" % entity["id"]))
        display(Markdown("\n".join(("* https://serlo.org/%s" % i for i in entity["injectedInto"]))))
        

# ID: 4701

Der Serlo-Inhalt https://serlo.org/4701 ist als gelöscht markiert, wird aber in folgenden anderen Inhalten eingebunden:

* https://serlo.org/36856

# ID: 4705

Der Serlo-Inhalt https://serlo.org/4705 ist als gelöscht markiert, wird aber in folgenden anderen Inhalten eingebunden:

...

Der Serlo-Inhalt https://serlo.org/136640 ist als gelöscht markiert, wird aber in folgenden anderen Inhalten eingebunden:

* https://serlo.org/132770

# ID: 136826

Der Serlo-Inhalt https://serlo.org/136826 ist als gelöscht markiert, wird aber in folgenden anderen Inhalten eingebunden:

* https://serlo.org/132783

# Liste inkludierter Aufgabensammlungen

In [53]:
for entity in entities.values():
    if entity["trashed"] == 0 and entity["type"] == "grouped-text-exercise" and len(entity["injectedInto"]) > 0:
        print("Id: %s" % entity["id"])
        print("Injected into: %s" % entity["injectedInto"])
        print()

Id: 2225
Injected into: [1619, 1773, 2173]

Id: 2233
Injected into: [1619, 1773]

Id: 2267
Injected into: [72262]

Id: 2303
Injected into: [1827]

Id: 2307
Injected into: [67468]

Id: 2345
Injected into: [149480]

Id: 2391
Injected into: [1797]

Id: 2403
Injected into: [1797, 109508]

Id: 2463
Injected into: [67468]

Id: 2505
Injected into: [2195]

Id: 2535
Injected into: [1797]

Id: 2569
Injected into: [1577]

Id: 2629
Injected into: [1797]

Id: 2669
Injected into: [1599]

Id: 2673
Injected into: [1599]

Id: 2677
Injected into: [1615]

Id: 2685
Injected into: [67463]

Id: 2697
Injected into: [1615]

Id: 2701
Injected into: [1615]

Id: 2725
Injected into: [30259]

Id: 2729
Injected into: [67463]

Id: 2739
Injected into: [1577]

Id: 2743
Injected into: [2173]

Id: 2781
Injected into: [19549, 30255]

Id: 2785
Injected into: [27803]

Id: 2801
Injected into: [51557]

Id: 2819
Injected into: [30250]

Id: 2851
Injected into: [51557]

Id: 2875
Injected into: [51557]

Id: 2883
Injected into: [